# 5. Objective Functions for Optimization Models

This notebook continues our step-by-step guide to creating optimization models. We are now at step 5 of our 7-step procedure:

1. ✓ Import Pyomo and Define the Sets
2. ✓ Define the Decision Variables
3. ✓ Define the Parameters
4. ✓ Define the Expressions
5. ➡️ Define the Objective Function
6. Define the Constraints

The objective function is perhaps the most crucial part of an optimization model - it defines what we are trying to achieve. Are we trying to maximize profit? Minimize cost? Optimize customer satisfaction? The objective function mathematically expresses these goals.

In this notebook, we will explore:
- What objective functions are and why they matter
- How to define different types of objectives in Pyomo
- Common objective function patterns
- Best practices for formulating objectives
- How to handle multiple objectives

## Setup: Creating Our Model Environment

Let's set up a production planning model example that we'll use throughout this notebook:

In [1]:
import pyomo.environ as pyo

# Create model
m = pyo.ConcreteModel()

# Sets
m.I = pyo.Set(initialize=['P1', 'P2', 'P3'])  # Products
m.J = pyo.Set(initialize=['M1', 'M2'])        # Machines

# Variables
m.x = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize={'P1': 100, 'P2': 150, 'P3': 200})  # Production quantities
m.y = pyo.Var(m.I, m.J, domain=pyo.Binary, initialize={(i, j): 1 for i in m.I for j in m.J})  # Machine assignments

# Parameters
m.p = pyo.Param(m.I, initialize={'P1': 100, 'P2': 150, 'P3': 200})  # Product prices
m.c = pyo.Param(m.J, initialize={'M1': 50, 'M2': 40})               # Machine costs
m.d = pyo.Param(m.I, initialize={'P1': 80, 'P2': 120, 'P3': 60})    # Demands

# Common expressions
m.revenue = pyo.Expression(expr=sum(m.p[i] * m.x[i] for i in m.I))
m.production_cost = pyo.Expression(expr=sum(m.c[j] * m.y[i,j] 
                                           for i in m.I for j in m.J))

## Understanding Objective Functions

An objective function in optimization represents what we're trying to optimize. It typically takes one of two forms:

1. Maximization objectives (e.g., maximize profit, utility, efficiency)
2. Minimization objectives (e.g., minimize cost, time, waste)

In Pyomo, we define objectives using the `Objective` component with a sense (maximize or minimize) and an expression. Here's a simple example:

In [2]:
# Simple profit maximization objective
m.profit = pyo.Objective(
    expr=m.revenue - m.production_cost,
    sense=pyo.maximize
)

## Common Objective Function Patterns

Let's look at several typical objective function patterns you might encounter in practice:

### 1. Cost Minimization

In [3]:
# 1. Cost Minimization
def cost_objective(m):
    """Total cost including production and machine setup"""
    production_cost = sum(m.c[j] * m.y[i,j] for i in m.I for j in m.J)
    setup_cost = 1000 * sum(m.y[i,j] for i in m.I for j in m.J)  # Fixed cost per setup
    return production_cost + setup_cost

m.objective = pyo.Objective(rule=cost_objective, sense=pyo.minimize)

Here, the goal is to minimize the cost. We can view the objective function by:

In [4]:
m.objective.pprint()

objective : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 50*y[P1,M1] + 40*y[P1,M2] + 50*y[P2,M1] + 40*y[P2,M2] + 50*y[P3,M1] + 40*y[P3,M2] + 1000*(y[P1,M1] + y[P1,M2] + y[P2,M1] + y[P2,M2] + y[P3,M1] + y[P3,M2])


We can access its value computed using the initial values of the variables and parameters by:

In [5]:
m.objective()

6270

The importance of variable and parameter initialization is that it allows us to see the objective function value computed using the initial values of the variables and parameters.


### 2. Revenue Maximization with Quality Bonus

We can create a new objective function that maximizes the revenue. Typically, we only have one objective function in the model, so we need to deactivate the previous objective function to avoid any conflict in the model. 

In [6]:
# Deactivate the previous objective before creating a new one
m.objective.deactivate()

Let's now create a new objective function that maximizes the revenue.

In [7]:
# 2. Revenue Maximization with Quality Bonus
m.quality_bonus = pyo.Param(m.I, initialize={'P1': 1.1, 'P2': 1.0, 'P3': 1.2})  # Quality multipliers
revenue_with_quality_bonus = sum(m.p[i] * m.quality_bonus[i] * m.x[i] for i in m.I)

m.max_revenue = pyo.Objective(expr=revenue_with_quality_bonus, sense=pyo.maximize)

We can view the objective function by:

In [8]:
print("Objective function: ", m.max_revenue.expr)
print("Objective function value: ", m.max_revenue())

Objective function:  110.00000000000001*x[P1] + 150.0*x[P2] + 240.0*x[P3]
Objective function value:  81500.0


In general, we can either maximize or minimize the objective function which can be set by the `sense` parameter.

## Writing Effective Objective Functions

When formulating objective functions, consider these key principles:

1. Clarity and Maintainability:
   - Use meaningful variable and parameter names
   - Break complex objectives into smaller expressions
   - Document the objective's components and purpose

2. Numerical Stability:
   - Keep objective terms in similar numerical ranges
   - Consider scaling factors for different components
   - Avoid unnecessary nonlinearities

Here's an example applying these principles:

In [9]:
#deactivate the previous objective function
m.max_revenue.deactivate()

In [10]:
# Max profit objective
m.base_revenue = pyo.Expression(expr=sum(m.p[i] * m.x[i] for i in m.I))
m.quality_premium = pyo.Expression(expr=sum(0.1 * m.p[i] * m.x[i] for i in m.I if m.quality_bonus[i] > 1.0))
m.total_production_cost = pyo.Expression(expr=sum(m.c[j] * m.y[i,j] for i in m.I for j in m.J))
m.total_setup_cost = pyo.Expression(expr=1000 * sum(m.y[i,j] for i in m.I for j in m.J))
m.total_profit = pyo.Expression(expr=m.base_revenue + m.quality_premium - m.total_production_cost - m.total_setup_cost)
m.profit_objective = pyo.Objective(expr=m.total_profit, sense=pyo.maximize)

print("Objective function: ", m.profit_objective.expr)
print("Objective function value: ", m.profit_objective())


Objective function:  total_profit
Objective function value:  71230.0


We can also achieve the same result by using Python functions. This is useful when we want to have a more complex objective function that is not easily expressible in a single expression.

In [11]:
def profit_function(m):
    """Calculate total profit with multiple components
    
    Components:
    1. Base revenue from product sales
    2. Quality premium for high-quality products
    3. Production costs from machine usage
    4. Fixed setup costs for machine configurations
    """
    # Revenue components
    base_revenue = sum(m.p[i] * m.x[i] for i in m.I)
    quality_premium = sum(0.1 * m.p[i] * m.x[i] 
                         for i in m.I if m.quality_bonus[i] > 1.0)
    
    # Cost components
    production_cost = sum(m.c[j] * m.y[i,j] 
                         for i in m.I for j in m.J)
    setup_cost = 1000 * sum(m.y[i,j] 
                           for i in m.I for j in m.J)
    
    # Print components for the first product for demonstration
    print("Objective components for P1:")
    print(f"- Base revenue: {m.p['P1']}*x[P1]")
    print(f"- Quality premium: {0.1 * m.p['P1']}*x[P1]")
    print(f"- Production cost: {m.c['M1']}*y[P1,M1] + {m.c['M2']}*y[P1,M2]")
    print("- Setup cost: 1000*y[P1,M1] + 1000*y[P1,M2]")
    print("\n")
    
    return base_revenue + quality_premium - production_cost - setup_cost

# Create and activate the structured objective
m.profit_with_function = pyo.Objective(rule=profit_function, sense=pyo.maximize)

print("Objective function: ", m.profit_with_function.expr)
print("Objective function value: ", m.profit_with_function())

Objective components for P1:
- Base revenue: 100*x[P1]
- Quality premium: 10.0*x[P1]
- Production cost: 50*y[P1,M1] + 40*y[P1,M2]
- Setup cost: 1000*y[P1,M1] + 1000*y[P1,M2]


Objective function:  100*x[P1] + 150*x[P2] + 200*x[P3] + 10.0*x[P1] + 20.0*x[P3] - (50*y[P1,M1] + 40*y[P1,M2] + 50*y[P2,M1] + 40*y[P2,M2] + 50*y[P3,M1] + 40*y[P3,M2]) - 1000*(y[P1,M1] + y[P1,M2] + y[P2,M1] + y[P2,M2] + y[P3,M1] + y[P3,M2])
Objective function value:  71230.0


## Handling Multiple Objectives

In real-world problems, we often have multiple competing objectives. There are several ways to handle this:

1. Weighted Sum Approach
2. Hierarchical Optimization
3. Goal Programming

Let's implement these approaches:

In [12]:
def weighted_objective(m):
    """Combine profit and service level objectives with weights"""
    # Normalize each component to similar scale
    max_possible_profit = sum(m.p[i] * m.d[i] for i in m.I)
    normalized_profit = (m.revenue - m.production_cost) / max_possible_profit
    
    service_levels = [(m.x[i] / m.d[i]) for i in m.I]
    avg_service = sum(service_levels) / len(m.I)
    
    # Weight the components
    profit_weight = 0.7
    service_weight = 0.3
    
    return profit_weight * normalized_profit + service_weight * avg_service

m.weighted_obj = pyo.Objective(rule=weighted_objective, sense=pyo.maximize)

print("Objective function: ", m.weighted_obj.expr)
print("Objective function value: ", m.weighted_obj())


Objective function:  0.7*(((100*x[P1] + 150*x[P2] + 200*x[P3]) - (50*y[P1,M1] + 40*y[P1,M2] + 50*y[P2,M1] + 40*y[P2,M2] + 50*y[P3,M1] + 40*y[P3,M2]))/38000) + 0.3*((0.0125*x[P1] + 0.008333333333333333*x[P2] + 0.016666666666666666*x[P3])/3)
Objective function value:  1.9138859649122808


The idea of the weighted sum approach is to combine the two objectives into a single objective function. The weights are used to balance the two objectives.

In [13]:
# Hierarchical Approach (using constraints)
# First ensure minimum service level
m.min_service_level = pyo.Constraint(
    m.I,
    rule=lambda m, i: m.x[i] >= 0.9 * m.d[i]  # Minimum 90% service level
)

# Then maximize profit within these constraints
m.hierarchical_obj = pyo.Objective(
    expr=m.revenue - m.production_cost,
    sense=pyo.maximize
)

print("Objective function: ", m.hierarchical_obj.expr)
print("Objective function value: ", m.hierarchical_obj())


Objective function:  (100*x[P1] + 150*x[P2] + 200*x[P3]) - (50*y[P1,M1] + 40*y[P1,M2] + 50*y[P2,M1] + 40*y[P2,M2] + 50*y[P3,M1] + 40*y[P3,M2])
Objective function value:  72230


The main idea of the hierarchical approach is to first ensure that the minimum service level is met and then maximize the profit within these constraints.

In [14]:
m.hierarchical_obj.deactivate()

def goal_programming_obj(m):
    """Minimize deviations from goals"""
    # Define goals
    profit_goal = 10000
    service_goal = 0.95
    
    # Calculate deviations
    profit_dev = abs((m.revenue - m.production_cost) - profit_goal)
    service_dev = sum(abs((m.x[i] / m.d[i]) - service_goal) for i in m.I)
    
    # Minimize total weighted deviation
    return profit_dev / profit_goal + service_dev / len(m.I)

m.goal_obj = pyo.Objective(rule=goal_programming_obj, sense=pyo.minimize)

print("Objective function: ", m.goal_obj.expr)
print("Objective function value: ", m.goal_obj())

Objective function:  abs((100*x[P1] + 150*x[P2] + 200*x[P3]) - (50*y[P1,M1] + 40*y[P1,M2] + 50*y[P2,M1] + 40*y[P2,M2] + 50*y[P3,M1] + 40*y[P3,M2]) - 10000)/10000 + (abs(0.0125*x[P1] - 0.95) + abs(0.008333333333333333*x[P2] - 0.95) + abs(0.016666666666666666*x[P3] - 0.95))/3
Objective function value:  7.217444444444444


The idea of the goal programming approach is to minimize the deviations from the goals. The goals are defined as the minimum profit and the minimum service level. The deviations are calculated as the absolute differences between the actual values and the goal values. The objective function is then the sum of the weighted deviations.


These are the most common approaches to handling multiple objectives in optimization models. The choice of approach depends on the specific problem and the goals of the decision-maker. 

In this class, we will focus on the weighted sum approach as it is the most straightforward and flexible approach.

## Conclusion

We've covered the key aspects of defining objective functions in Pyomo optimization models:

- What objective functions are and why they matter
- How to define different types of objectives in Pyomo
- Common objective function patterns
- Best practices for formulating objectives
- How to handle multiple objectives

As you can see, objective functions are essentially just expressions, with an additional sense parameter (maximize or minimize). However, the way we construct these expressions can have a significant impact on the model's performance and the quality of the solutions we obtain. 

In the next notebook, we will learn how to add constraints to the model.
